In [214]:
import matplotlib.pyplot as plt
import matplotlib.image as npimg 
import numpy as np
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

In [215]:
def YellowAndWhite(hsv_image): #Done
    masks = []
    low_white,high_white = np.array([0, 0, 200]),np.array([180, 25, 255])
    low_yellow,high_yellow = np.array([15, 40, 205]),np.array([25, 255, 255])
    white = cv2.inRange(hsv_image, low_white, high_white)
    masks.append(white)
    yellow = cv2.inRange(hsv_image, low_yellow, high_yellow)
    masks.append(yellow)
    return cv2.add(*masks)

In [216]:
def Force3D(mask, image): #Done
    masked_image = np.zeros_like(image)
    for i in range(3): 
        masked_image[:,:,i] = mask.copy()
    return masked_image

In [217]:
def RegionOfInterest(image, points): #Done
    mask = np.zeros_like(image)   
    ignore_color = 255

    cv2.fillPoly(mask, [points],ignore_color)
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image

In [218]:
def LaneX(leftx,rightx):   
    global maxr, minr, maxl, minl
    maxr.append(max(rightx))
    minr.append(min(rightx))
    maxl.append(max(leftx))
    minl.append(min(leftx))
    if len(maxr)>100:
        maxr.pop(0)
    if len(minr)>100:    
        minr.pop(0)
    if len(maxl)>100:    
        maxl.pop(0)
    if len(minl)>100:    
        minl.pop(0)

In [223]:
def LaneLineDetector(original_image):
    image = np.copy(original_image)
    width,height = len(image[0]),len(image)
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    color_mask = YellowAndWhite(hsv_image)

    masked_image = Force3D(color_mask, image) 

    edge_mask = cv2.Canny(masked_image, 50, 150) #check if you even need this. The mask may be good enough or better
      
    edged_image = Force3D(edge_mask, image) 
    
    p1,p2,p3,p4  = [10,height-10],[int(29/60*width),int(9/16*height)],[int(31/60*width),int(9/16*height)],[width-10,height-10]
    points = np.array([p1,p2,p3,p4], dtype=np.int32) 
    region_image= RegionOfInterest(edged_image, points) 
    
    grey_image = cv2.cvtColor(region_image, cv2.COLOR_RGB2GRAY)
    plt.imshow(grey_image)
    
     #9
    rho = 1
    theta = np.pi/180
    threshold = 35
    min_line_length = 40
    max_line_gap = 15
    
    lines = cv2.HoughLinesP(grey_image, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    
    global left_lane_line, right_lane_line
    lined_image = np.zeros_like(image) 
    
    for line in lines:
        for x1,y1,x2,y2 in line: #No lines with little slope. No lines that cross the middle line.
            if x1>width/2 and x2>width/2:
                if (y2-y1)/(x2-x1) > 0.4: 
                    cv2.line(lined_image, (x1, y1), (x2, y2), [255,0,0], 5)
                    right_lane_line.append([x1,y1])
                    right_lane_line.append([x2,y2])
            elif x1<width/2 and x2<width/2:
                if (y2-y1)/(x2-x1) < -0.4: 
                    cv2.line(lined_image, (x1, y1), (x2, y2), [255,0,0], 5)
                    left_lane_line.append([x1,y1])
                    left_lane_line.append([x2,y2])        
    
    while len(left_lane_line)> 50:
        left_lane_line.pop(0)
        
    while len(right_lane_line)> 50:
        right_lane_line.pop(0)
    
    leftx,lefty = [point[0] for point in left_lane_line],[point[1] for point in left_lane_line]
    leftmxb = np.polyfit(leftx,lefty,1)
    
    rightx,righty = [point[0] for point in right_lane_line],[point[1] for point in right_lane_line]
    rightmxb = np.polyfit(rightx,righty,1)
    
    fl = np.poly1d(leftmxb)
    fr = np.poly1d(rightmxb)
    
    LaneX(leftx,rightx)
        
    #lx1,lx2,rx1,rx2 = int(10),int(width/2-10),int(width/2+10),int(width-10)
    lx1,lx2,rx1,rx2 = int(np.mean(minl)),int(np.mean(maxl)),int(np.mean(minr)),int(np.mean(maxr))
    ly1,ly2,ry1,ry2 = int(fl(lx1)),int(fl(lx2)),int(fr(rx1)),int(fr(rx2))
    
    cv2.line(lined_image, (lx1,ly1), (lx2, ly2), [0,0,255], 10) 
    cv2.line(lined_image, (rx1,ry1), (rx2, ry2), [0,0,255], 10)
    

    laned_image = cv2.addWeighted(original_image, 0.8, lined_image, 1, 0) 

    return laned_image 

In [224]:
global left_lane_line, right_lane_line, maxr, minr, maxl, minl
left_lane_line, right_lane_line, maxr, minr, maxl, minl = [],[],[],[],[],[]
final_clip_output = 'challenge.mp4'
clip = VideoFileClip("TestVideos/challenge.mp4")
final_clip = clip.fl_image(LaneLineDetector)
%time final_clip.write_videofile(final_clip_output, audio=False)

[MoviePy] >>>> Building video challenge2.mp4
[MoviePy] Writing video challenge2.mp4


100%|██████████| 251/251 [00:47<00:00,  5.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge2.mp4 

CPU times: user 1min 31s, sys: 7.72 s, total: 1min 39s
Wall time: 50.3 s


In [225]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(final_clip_output))

In [226]:
white_clip_output = 'solidWhiteRight.mp4'
clip1 = VideoFileClip("TestVideos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(LaneLineDetector)
%time white_clip.write_videofile(white_clip_output, audio=False)

[MoviePy] >>>> Building video solidWhiteRight2.mp4
[MoviePy] Writing video solidWhiteRight2.mp4


100%|█████████▉| 221/222 [00:23<00:00,  9.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidWhiteRight2.mp4 

CPU times: user 50.8 s, sys: 3.94 s, total: 54.8 s
Wall time: 25.2 s


In [227]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_clip_output))

In [228]:
yellow_clip_output = 'solidYellowLeft.mp4'
clip2 = VideoFileClip("TestVideos/solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(LaneLineDetector)
%time yellow_clip.write_videofile(yellow_clip_output, audio=False)

[MoviePy] >>>> Building video solidYellowLeft2.mp4
[MoviePy] Writing video solidYellowLeft2.mp4


100%|█████████▉| 681/682 [01:15<00:00,  9.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidYellowLeft2.mp4 

CPU times: user 2min 30s, sys: 11 s, total: 2min 40s
Wall time: 1min 17s


In [229]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_clip_output))